In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install wikibase-api
!{sys.executable} -m pip install sparqlwrapper
import json
import csv
from wikibase_api import Wikibase
from SPARQLWrapper import SPARQLWrapper, JSON

wb = Wikibase(
    api_url='https://doku.wikibase.wiki/w/api.php', 
    oauth_credentials=None, 
    login_credentials={'bot_username':'Bauert@dokubot', 'bot_password':'valr43kp7ajnalke81lmtoa0ph34unl4'}, 
    is_bot=True, 
    summary='Modified using wikibase-api for Python', 
    config_path=None)

endpoint_url = "https://doku.wikibase.wiki/query/proxy/wdqs/bigdata/namespace/wdq/sparql"

def get_results_from_sparql_endpoint(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


In [2]:
query_all_possible_datafields = """PREFIX prop: <https://doku.wikibase.wiki/prop/direct/>
PREFIX item: <https://doku.wikibase.wiki/entity/>

SELECT DISTINCT ?dataField ?dataFieldLabel # ?definition ?coding ?subfields ?subfieldsLabel ?dataFieldType ?validation ?dataFieldAltLabel
WHERE { 
#  ?dataField wikibase:propertyType ?dataFieldType .
  ?dataField prop:P2 item:Q2 .
#    ?dataField rdfs:label ?dataFieldLabel .
#  FILTER(STRSTARTS(?dataFieldLabel, 'Satz')) .
#  OPTIONAL{ ?dataField prop:P1 ?definition }
#  OPTIONAL{ ?dataField prop:P9 ?validation }
#  OPTIONAL{ ?dataField prop:P4 ?coding }
#  OPTIONAL{ ?dataField prop:P15 ?subfields }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" }
}
ORDER BY ASC(xsd:integer(STRAFTER(STR(?statementValue), 'Q')))

"""
results = get_results_from_sparql_endpoint(endpoint_url, query_all_possible_datafields)
header = results["head"]["vars"]
data = results["results"]["bindings"]
data_field_paths = []
data_field_labels = []
for item in data:
    # print('item: ', item["dataField"]["value"])
    data_field_paths.append(item["dataField"]["value"])
    data_field_labels.append(item["dataFieldLabel"]["value"])
print(len(data_field_paths))
print(len(data_field_labels))
print(data_field_paths)
print(data_field_labels)

    
#print(type(results), results)
#data_fields = data["dataField"]
#print(type(data), data)
#print(type(data_fields), data_fields)
#print(type(header), header)

46
46
['https://doku.wikibase.wiki/entity/P103', 'https://doku.wikibase.wiki/entity/P104', 'https://doku.wikibase.wiki/entity/P105', 'https://doku.wikibase.wiki/entity/P106', 'https://doku.wikibase.wiki/entity/P107', 'https://doku.wikibase.wiki/entity/P108', 'https://doku.wikibase.wiki/entity/P133', 'https://doku.wikibase.wiki/entity/P160', 'https://doku.wikibase.wiki/entity/P171', 'https://doku.wikibase.wiki/entity/P53', 'https://doku.wikibase.wiki/entity/P55', 'https://doku.wikibase.wiki/entity/P56', 'https://doku.wikibase.wiki/entity/P58', 'https://doku.wikibase.wiki/entity/P59', 'https://doku.wikibase.wiki/entity/P63', 'https://doku.wikibase.wiki/entity/P65', 'https://doku.wikibase.wiki/entity/P66', 'https://doku.wikibase.wiki/entity/P68', 'https://doku.wikibase.wiki/entity/P69', 'https://doku.wikibase.wiki/entity/P70', 'https://doku.wikibase.wiki/entity/P71', 'https://doku.wikibase.wiki/entity/P72', 'https://doku.wikibase.wiki/entity/P73', 'https://doku.wikibase.wiki/entity/P80', 

In [ ]:
r = wb.entity.get(data_field_paths[0])
s = wb.entity.search('PICA', 'en', entity_type='item', limit=300)
#print(r)
# Pretty Print JSON
json_formatted_str = json.dumps(r, indent=4)
print(json_formatted_str)

In [ ]:
# create CSV file to store results
# now we will open a file for writing
data_file = open('data_file.csv', 'w')
 
# create the csv writer object
csv_writer = csv.writer(data_file)
 
# Counter variable used for writing
# headers to the CSV file
count = 0
 
for entity in data:
    if count == 0:
 
        # Writing headers of CSV file
        header = entity.keys()
        csv_writer.writerow(header)
        count += 1
 
    # Writing data of CSV file
    csv_writer.writerow(entity.values())
 
data_file.close()